# CLIPtionary Attack Notebook
<h2>License</h2>
Copyright (c) 2021 Bearsharktopusdev

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# HOW TO USE
<h2>Upload your image in the file thing. Put the image path in Parameters as indicated. Fill in other parameters as desired. Restart + Run All and then go get some coffee because it'll take a little bit.</h2>

# Imports & Shit

In [1]:
#@markdown
!nvidia-smi

Sat Aug 28 22:23:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    48W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@markdown
%cd /content/

!git clone https://github.com/openai/CLIP.git

%cd /content/CLIP/

!pip install ftfy
!pip install tqdm
from tqdm import trange

import imageio
import torchvision

from IPython import display
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import clip
!pip install kornia
import kornia
import torch
import torch.nn.functional as F

from PIL import Image

import random

clip.available_models()

#RN50x4
#ViT-B/32

import numpy as np

# Load the model
perceptor, preprocess = clip.load('ViT-B/32', jit=True) # usually use vit
perceptor = perceptor.eval()

clear_output()

In [3]:
#@markdown

%cd /content/
!git clone https://github.com/first20hours/google-10000-english

clear_output()

In [4]:
#@markdown
file = open("/content/google-10000-english/google-10000-english-usa-no-swears-short.txt", "r")
short_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/google-10000-english-usa-no-swears-medium.txt", "r")
medium_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/google-10000-english-usa-no-swears-long.txt", "r")
long_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/20k.txt", "r")
full_list = file.read().splitlines()
file.close()

# Parameters

In [5]:
check_amount = 500 #100, number of terms to contest per epoch
seed_epochs = 10 #10, number of iterations of the checking algorithm for the initial seeding
self_comp_epochs = 4 #4, number of iterations of the self-comparative attack
culling_threshold = 1.0 #1.2, any phrase worth less than this many words is culled from the list. 
# for example, with a threshold of 1 and 100 words, any word that scores less than 0.01 (1/100) is culled
# Recommended to be kept between 1 and 2

seed_words = "giant, tall, grotesque, sirenhead, daytime, thin, slender, slimyswampghost, photo" #"", comma separated list of words to seed the initial epochs with.
image = "/content/sirenhead.jpg" #don't delete /content/, just upload to google colab and put your filename there

dictionary_from_top = 5 #Adds this many words from the top of the frequency-rated dictionary to the dictionary attack
dictionary_from_random = 5 #Adds this many words randomly from the frequency-rated dictionary to the dictionary attack

iterative_guess_words = "sirenhead, photograph of, slimyswampghost, sirenhead, slenderman" #"", comma separated list of words for the iterative guess & check step

# Definitions

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)

image = preprocess(Image.open(image)).unsqueeze(0).to(device)
image_features = model.encode_image(image)

winner_list = []

def rate_words(input_list):
  print("rating")
  text = clip.tokenize(input_list).to(device)

  with torch.no_grad():
      text_features = model.encode_text(text)
      
      logits_per_image, logits_per_text = model(image, text)
      probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  probs_list = probs.tolist()[0]
  input_probs_zip = zip(input_list, probs_list)
  input_probs_list = []
  for i in input_probs_zip:
    input_probs_list.append(i)
  result = [i for i in input_probs_list if i[1] >= max(culling_threshold/len(input_list), 0.5/check_amount)]
  result = remove_duplicates(result)
  result.sort(key=lambda x:x[1], reverse=True)

  return result

def refill_list(input_list):
  random_list = random.sample(full_list, check_amount)
  while len(input_list) < check_amount:
    input_list.append(random_list[0])
    random_list.pop(0)

def remove_duplicates(input_list):
    return list(set([i for i in input_list]))

def attack(input_list):
  for k in input_list:
    global winner_list
    winner_list = remove_duplicates(winner_list)
    thing = [x[0] for x in winner_list]
    thinglen = len(thing)
    thinglenlen = len(str.split(thing[0]))+1
    new_list = [thing]*thinglenlen
    new_new_list = []
    for i in range(thinglenlen):
      for j in new_list[i]:
        newj = str.split(j)
        newj.insert(i, k)
        newj = " ".join(newj)
        new_new_list.append(newj)
    for i in thing:
      new_new_list.append(i)
    new_new_list.append(k)

    winner_list = rate_words(new_new_list)

    print('\n---\nprinting top 10')
    winner_list.sort(key=lambda x:x[1], reverse=True)
    for i in range(len(winner_list)):
      if i < 10:
        print(winner_list[i])

# Dictionary Seeding

In [7]:
seed_word_list = seed_words.split(", ")
initial_attack = full_list[0:check_amount - len(seed_word_list)]
initial_attack = initial_attack + seed_word_list

for i in range(seed_epochs):
  refill_list(initial_attack)
  winner_list = rate_words(initial_attack)
  initial_attack = [x[0] for x in winner_list]

  print('\n---')
  for i in winner_list:
    print(i)

rating

---
('slimyswampghost', 0.8798828125)
('slender', 0.015869140625)
('him', 0.004268646240234375)
('long', 0.004138946533203125)
('giant', 0.0038280487060546875)
('top', 0.00302886962890625)
('he', 0.0029354095458984375)
('tall', 0.00225067138671875)
('end', 0.002017974853515625)
rating

---
('slimyswampghost', 0.9091796875)
('slender', 0.0164031982421875)
('him', 0.004413604736328125)
('long', 0.004276275634765625)
('giant', 0.003955841064453125)
('top', 0.0031299591064453125)
('he', 0.003032684326171875)
('titan', 0.002513885498046875)
('tall', 0.0023250579833984375)
('end', 0.0020847320556640625)
rating

---
('slimyswampghost', 0.900390625)
('slender', 0.0162353515625)
('him', 0.004367828369140625)
('long', 0.00423431396484375)
('giant', 0.003917694091796875)
('tripod', 0.003795623779296875)
('top', 0.0030975341796875)
('he', 0.003002166748046875)
('articulated', 0.003002166748046875)
('titan', 0.0024890899658203125)
('tall', 0.0023021697998046875)
('end', 0.002063751220703125

# Self-Comparing Attack

In [8]:
self_comp_attack = []
temp = []

new = [x[0] for x in winner_list]

for j in range(self_comp_epochs):
  if j == 0:
    for k in new:
      for i in new:
        self_comp_attack.append(k + " " + i)
      self_comp_attack.append(k)
  else:
    for k in new:
      for i in temp:
        self_comp_attack.append(k + " " + i)

  winner_list = rate_words(self_comp_attack)
  self_comp_attack = [x[0] for x in winner_list]
  temp = [x[0] for x in winner_list]

  print('\n---\nprinting top 10')
  for i in range(len(winner_list)):
    if i < 10:
      print(winner_list[i])

rating

---
('slimyswampghost garrison', 0.1595458984375)
('slimyswampghost tall', 0.062469482421875)
('tripod slimyswampghost', 0.056884765625)
('slimyswampghost titan', 0.05511474609375)
('slimyswampghost tripod', 0.050201416015625)
('slimyswampghost bone', 0.04864501953125)
('terrestrial slimyswampghost', 0.04034423828125)
('slimyswampghost giant', 0.031890869140625)
('slimyswampghost upright', 0.031402587890625)
('garrison slimyswampghost', 0.0309295654296875)
('titan slimyswampghost', 0.029052734375)
('slender slimyswampghost', 0.025238037109375)
('tall slimyswampghost', 0.022979736328125)
('slimyswampghost terrestrial', 0.0226287841796875)
('slimyswampghost slender', 0.0219268798828125)
('bone slimyswampghost', 0.0196533203125)
('obituaries slimyswampghost', 0.0190582275390625)
('ack slimyswampghost', 0.0178985595703125)
('slimyswampghost kansas', 0.0157928466796875)
('slimyswampghost obituaries', 0.01438140869140625)
('upright slimyswampghost', 0.0124969482421875)
('kansas slimy

# Iterative Dictionary Attack

In [9]:
dictionary_attack = full_list[0:dictionary_from_top]
random_list = random.sample(full_list, dictionary_from_random)
dictionary_attack = dictionary_attack + random_list

attack(dictionary_attack)

rating

---
printing top 10
('tall ack bone titan tripod the terrestrial terrestrial slimyswampghost', 0.0021190643310546875)
('tall tall ack bone titan tripod the terrestrial terrestrial slimyswampghost', 0.00186920166015625)
('he he he he the tall terrestrial bone terrestrial slimyswampghost garrison', 0.001811981201171875)
('he he he he he the tall terrestrial bone terrestrial slimyswampghost garrison', 0.001811981201171875)
('he he he he he tall ack bone the terrestrial slimyswampghost garrison', 0.0017023086547851562)
('he he he he tall ack bone the terrestrial slimyswampghost garrison', 0.0017023086547851562)
('kansas terrestrial bone titan the kansas tripod he terrestrial slimyswampghost garrison', 0.0016498565673828125)
('he he he he he tall ack the bone terrestrial slimyswampghost garrison', 0.0015993118286132812)
('giant tall ack bone titan tripod the terrestrial terrestrial slimyswampghost', 0.0015993118286132812)
('articulated giant tall ack bone articulated tripod the terr

# Synonym/Antonym Attack

# Iterative Guess & Check

In [10]:
words_list = iterative_guess_words.split(", ")

attack(words_list)

rating

---
printing top 10
('tall ack the bone titan tripod of terrestrial and terrestrial slimyswampghost', 0.022491455078125)
('tall ack the bone titan tripod of terrestrial rules and terrestrial slimyswampghost', 0.0218048095703125)
('the tall ack bone titan tripod of terrestrial rules and terrestrial slimyswampghost', 0.0204925537109375)
('tall ack the bone titan of tripod and terrestrial terrestrial to slimyswampghost', 0.0198516845703125)
('tall ack the bone titan tripod of terrestrial terrestrial slimyswampghost', 0.0198516845703125)
('tall ack the bone titan of tripod and terrestrial to terrestrial slimyswampghost', 0.0192413330078125)
('tall ack bone the titan tripod of terrestrial rules and terrestrial slimyswampghost', 0.0192413330078125)
('tall ack bone the titan tripod of terrestrial and terrestrial slimyswampghost', 0.0192413330078125)
('tall ack the bone titan of tripod to terrestrial rules and terrestrial slimyswampghost', 0.0192413330078125)
('the tall ack bone titan 